In [23]:
import uuid
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.sql import func
from sqlalchemy.orm import sessionmaker
from alchemy import Referee, Base, Proposal, Review, ScoreQuantile
import lorem
import json
from shutil import copyfile

In [24]:
with open('credentials.json') as f:
    credentials = json.load(f)
runEnv = credentials['env'] # development = sqlite; production = mysql
dsn = credentials[runEnv]['dbo'] # in the format <engine>://<connection_string>

In [25]:
if runEnv == 'test':
    !rm dt_opc_test.db
else:
    engine = create_engine(dsn)
    connection = engine.connect()
    connection.execute('drop table if exists reviews')
    connection.execute('drop table if exists proposals')
    connection.execute('drop table if exists referees')
    connection.execute('drop table if exists review_rating')
    connection.execute('drop table if exists score_quantiles')
    connection.close()
!rm proposals/*.pdf

In [26]:
## generate 50 referees
ref_num = 50

referee_ids = [str(uuid.uuid4()).split('-')[0] for i in range(50)]

## generate 50 proposals

prop_num = 50
prop_idx = np.arange(1000)
np.random.shuffle(prop_idx)
proposal_ids = ['103.x-{0:04d}'.format(rand_id) for rand_id in prop_idx[:50]]
proposal_titles = [lorem.sentence() for i in range(50)]
proposal_abstract = [lorem.paragraph() for i in range(50)]

In [27]:
# an Engine, which the Session will use for connection
# resources
#engine = create_engine('sqlite:///dt_opc_test.db')
engine = create_engine(dsn)

# create a configured "Session" class
Session = sessionmaker(bind=engine)

# create a Session
session = Session()

In [28]:
Base.metadata.create_all(engine)

In [29]:
for i, ref_id in enumerate(referee_ids):
    referee = Referee(uuid=ref_id)
    referee.proposal_submitted_id = i+1
    session.add(referee)

In [30]:
for i, prop_id in enumerate(proposal_ids):
    session.add(Proposal(eso_id=prop_id, title=proposal_titles[i], 
                         abstract=proposal_abstract[i]))
    # make a dummy pdf file:
    copyfile('proposals/placeholder.pdf.master', 'proposals/'+prop_id+'.pdf')    

In [31]:
session.commit()

In [32]:
for ref in session.query(Referee):
    for prop in np.random.choice(session.query(Proposal).all(), size=8).tolist():
        ref.proposals.append(prop)

In [33]:
session.commit()

In [34]:
np.random.seed = 25081980
for ref in session.query(Referee):
    # 90% of referees are finished, for round 2
    if np.random.random() > 0.1:
        ref.accepted_tou = True
        ref.finalized_submission = True
        for review in ref.reviews:
            review.comment = lorem.paragraph()
            review.score = np.random.uniform(1, 5)
            review.ref_knowledge = np.random.randint(3) + 1
            if np.random.random() > 0.8:
                review.conflicted = np.random.randint(2) + 1
    else:
        ref.accepted_tou = False
        
    # 20% of referees are finished
#    if np.random.random() > 0.8: 
#        ref.accepted_tou = True
#        for review in ref.reviews:
#            review.comment = lorem.paragraph()
#            review.score = np.random.uniform(1, 5)
#            review.ref_knowledge = np.random.randint(3) + 1
#            if np.random.random() > 0.8:
#                review.conflicted = np.random.randint(2) + 1
#        ref.finalized_submission = True
    # 60% have accepted tou and done some work; remaining 20% haven't accepted tou or done work yet
#    elif np.random.random() > 0.2:
#        ref.accepted_tou = True
#        for review in ref.reviews:
#            if np.random.random() > 0.5:
#                review.comment = lorem.paragraph()
#            if np.random.random() > 0.5:
#                review.score = np.random.uniform(-1, 7)
#            if np.random.random() > 0.5:
#                review.ref_knowledge = np.random.randint(3) + 1
#            if np.random.random() > 0.8:
#                review.conflicted = np.random.randint(2) + 1


In [35]:
session.commit()


In [36]:
# calculate the avg score for each proposal
proposal_score_averages = session.query(func.avg(Review.score).label('avg')).group_by(Review.proposal_id).all()
df = pd.DataFrame(proposal_score_averages)

# calculate the score quartiles and store them in the db
quartiles = df.quantile([0.25, 0.5, 0.75])
for i, q in quartiles.itertuples():
    score_quantile = ScoreQuantile(quantile_name=i, score=q)
    session.add(score_quantile)
session.commit()

In [37]:
if runEnv == 'test':
    !cp dt_opc_test.db dt_opc_test.db.bak

In [38]:
test_ref = session.query(Referee).first()